<b>PART 1
    Install libraries for wrangling

In [1]:
%conda install BeautifulSoup4
%conda install requests

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.0

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - beautifulsoup4


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.11.28         |           py36_0         156 KB
    beautifulsoup4-4.8.1       |           py36_0         153 KB
    soupsieve-1.9.5            |           py36_0          61 KB
    openssl-1.1.1d             |       h7b6447c_3         3.7 MB
    ------------------------------------------------------------
                                           Total:         4.1 MB

The following NEW packages will be INSTALLED:

    soupsieve:      1.9.5-py36_0                 

The following packages will b

<b>Other necessary libraries for maps

In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


<b>DATA wrangling begins 

In [3]:
from bs4 import BeautifulSoup
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
soup = BeautifulSoup(website_url.text,'html.parser')

<b>Text selected from wikipedia table, added to a dataframe, entries with borough as not assigned are dropped

In [4]:
#https://youtu.be/cdAYhLSXDj4 - reference website
table = soup.find('table',{'class':'wikitable sortable'}).tbody
rows = table.find_all('tr')
columns = [v.text.replace('\n', '') for v in rows[0].find_all('th')]
df = pd.DataFrame(columns = columns)
for i in range (1,len(rows)):
    tds = rows[i].find_all('td')
    values = [td.text.replace('\n','') for td in tds]
    df = df.append(pd.Series(values, index = columns), ignore_index = True)
#https://stackoverflow.com/questions/20490274/how-to-reset-index-in-a-pandas-dataframe
df1 = df.drop(df[df['Borough'] == 'Not assigned'].index).reset_index(drop=True) 
df1.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor


<b>join neighborhoods that have the same postal code 

In [60]:
df2 = df1.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(','.join).to_frame().reset_index()
df2.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


<b>Find which index has borough as not null and neighbourhood as null, make the necessary update

In [96]:
df2[(df2['Borough'] != 'null') & (df2['Neighbourhood'] == 'Not assigned')]
df2.at[85,'Neighbourhood'] = df2.loc[85,'Borough']
df2.shape


(103, 3)

<b>PART 2
    Finding the lat and long of each neighbourhood

In [94]:
geo = pd.read_csv('Geospatial_Coordinates.csv')

In [98]:
df2_geo = pd.merge(left=df2,right=geo, left_on='Postcode', right_on='Postal Code')

In [101]:
df2_geo.drop('Postal Code', axis=1).head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


<b>Getting the geo coordinates of Toronto

In [108]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto City are 43.653963, -79.387207.


<b>create map of Toronto using latitude and longitude values

In [112]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df2_geo['Latitude'], df2_geo['Longitude'], df2_geo['Borough'], df2_geo['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

#### PART 3
Define Foursquare Credentials and Version

In [116]:
CLIENT_ID = 'LVFAIQW4DAICZN0BXPEM5AVDE3K34A53NYR10H21TRDMWXNY' # your Foursquare ID
CLIENT_SECRET = 'QTOIRFKYIWNZOVNVUWVI4WAYAD3IVTPI4GIXW2OXROGAC5HI' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: LVFAIQW4DAICZN0BXPEM5AVDE3K34A53NYR10H21TRDMWXNY
CLIENT_SECRET:QTOIRFKYIWNZOVNVUWVI4WAYAD3IVTPI4GIXW2OXROGAC5HI


From the Foursquare lab in the previous module, we know that all the information is in the *items* key. Before we proceed, let's borrow the **get_category_type** function from the Foursquare lab.

In [117]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

#### Let's create a function to explore all the neighborhoods in Manhattan

In [126]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
    
    # return a list first, then transform the entire list to a dataframe. 
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    
    return(nearby_venues)

In [127]:
Toronto_venues = getNearbyVenues(names=df2_geo['Neighbourhood'],
                                   latitudes=df2_geo['Latitude'],
                                   longitudes=df2_geo['Longitude']
                                  )

In [128]:
Toronto_venues.head()

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge,Malvern",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,"Rouge,Malvern",43.806686,-79.194353,Interprovincial Group,43.805630,-79.200378,Print Shop
2,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497,Chris Effects Painting,43.784343,-79.163742,Construction & Landscaping
3,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
4,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497,Affordable Toronto Movers,43.787919,-79.162977,Moving Target


### Analyze Each Neighborhood

In [129]:
# one hot encoding
Toronto_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")
#print(Toronto_onehot)
# add neighborhood column back to dataframe
Toronto_onehot['Neighbourhood'] = Toronto_venues['Neighbourhood'] 
Toronto_onehot.set_index('Neighbourhood', inplace=True)
Toronto_onehot.head()

,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Carpet Store,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Festival,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,Fraternity House,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Market,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Moving Target,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Piano Bar,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Soccer Field,Social Club,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
Neighbourhood,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [132]:
Toronto_grouped = Toronto_onehot.groupby('Neighbourhood').sum().reset_index()
Toronto_grouped.head()

,Neighbourhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Carpet Store,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Festival,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,Fraternity House,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Market,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Moving Target,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Piano Bar,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Soccer Field,Social Club,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Adelaide,King,Richmond",0,0,0,0,0,0,0,0,2,0,0,0,1,0,3,0,0,0,0,0,0,3,0,3,0,0,0,0,0,0,0,0,0,0,0

#### Let's print each neighborhood along with the top 5 most common venues

In [134]:
num_top_venues = 5

for hood in Toronto_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = Toronto_grouped[Toronto_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide,King,Richmond----
             venue  freq
0      Coffee Shop   7.0
1             Café   5.0
2       Steakhouse   4.0
3  Thai Restaurant   4.0
4           Bakery   3.0


----Agincourt----
                       venue  freq
0                     Lounge   1.0
1             Breakfast Spot   1.0
2  Latin American Restaurant   1.0
3               Skating Rink   1.0
4             Clothing Store   1.0


----Agincourt North,L'Amoreaux East,Milliken,Steeles East----
                 venue  freq
0                 Park   1.0
1           Playground   1.0
2  Arts & Crafts Store   1.0
3    Accessories Store   0.0
4   Mexican Restaurant   0.0


----Albion Gardens,Beaumond Heights,Humbergate,Jamestown,Mount Olive,Silverstone,South Steeles,Thistletown----
                  venue  freq
0   Fried Chicken Joint   1.0
1         Grocery Store   1.0
2              Pharmacy   1.0
3  Fast Food Restaurant   1.0
4           Pizza Place   1.0


----Alderwood,Long Branch----
          venue  freq
0   

#### Let's put that into a *pandas* dataframe

In [136]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [137]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = Toronto_grouped['Neighbourhood']

for ind in np.arange(Toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Coffee Shop,Café,Thai Restaurant,Steakhouse,Bar,Bakery,Restaurant,Salad Place,Sushi Restaurant,Asian Restaurant
1,Agincourt,Latin American Restaurant,Skating Rink,Lounge,Clothing Store,Breakfast Spot,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run
2,"Agincourt North,L'Amoreaux East,Milliken,Steel...",Park,Arts & Crafts Store,Playground,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Donut Shop
3,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",Grocery Store,Fried Chicken Joint,Fast Food Restaurant,Pharmacy,Video Store,Beer Store,Pizza Place,Sandwich Place,Dog Run,Dessert Shop
4,"Alderwood,Long Branch",Pizza Place,Skating Rink,Coffee Shop,Gym,Pub,Pharmacy,Dance Studio,Sandwich Place,Discount Store,Department Store


### Cluster Neighborhoods

In [ ]:
# set number of clusters
kclusters = 5

Toronto_grouped_clustering = Toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

### Adding cluster labels to dataframe

In [141]:
#Toronto_data
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
neighborhoods_venues_sorted.head()

,Cluster Labels,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,1,"Adelaide,King,Richmond",Coffee Shop,Café,Thai Restaurant,Steakhouse,Bar,Bakery,Restaurant,Salad Place,Sushi Restaurant,Asian Restaurant
1,2,Agincourt,Latin American Restaurant,Skating Rink,Lounge,Clothing Store,Breakfast Spot,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run
2,2,"Agincourt North,L'Amoreaux East,Milliken,Steel...",Park,Arts & Crafts Store,Playground,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Donut Shop
3,2,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",Grocery Store,Fried Chicken Joint,Fast Food Restaurant,Pharmacy,Video Store,Beer Store,Pizza Place,Sandwich Place,Dog Run,Dessert Shop
4,2,"Alderwood,Long Branch",Pizza Place,Skating Rink,Coffee Shop,Gym,Pub,Pharmacy,Dance Studio,Sandwich Place,Discount Store,Department Store


### Lets check the size of the 2 data frames

In [144]:
print(df2_geo.shape)
print(neighborhoods_venues_sorted.shape)

(103, 6)
(101, 12)


### We see that df2_geo has 2 extra entries. Let's drop them

In [147]:
list1= df2_geo['Neighbourhood'].values.tolist()
list2= neighborhoods_venues_sorted['Neighbourhood'].values.tolist()

for i in list1:
    if i not in list2:
        print(i)

Upper Rouge


In [157]:
df2_geo[df2_geo['Neighbourhood'] == 'Queen\'s Park']


,Postcode,Borough,Neighbourhood,Postal Code,Latitude,Longitude
85,M7A,Queen's Park,Queen's Park,M7A,43.662301,-79.389494
93,M9A,Downtown Toronto,Queen's Park,M9A,43.667856,-79.532242


In [156]:
df2_geo[df2_geo['Neighbourhood'] == 'Upper Rouge']

,Postcode,Borough,Neighbourhood,Postal Code,Latitude,Longitude
16,M1X,Scarborough,Upper Rouge,M1X,43.836125,-79.205636


In [158]:
df2_geo_clean = df2_geo.drop([16,93], axis=0).reset_index()

### Merge the frequency data with Toronto geo coordinates data

In [159]:
Toronto_merged = df2_geo_clean

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Toronto_merged = Toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

Toronto_merged.head() # check the last columns!

,index,Postcode,Borough,Neighbourhood,Postal Code,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,0,M1B,Scarborough,"Rouge,Malvern",M1B,43.806686,-79.194353,2,Fast Food Restaurant,Print Shop,Yoga Studio,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Donut Shop
1,1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",M1C,43.784535,-79.160497,2,Moving Target,Construction & Landscaping,Bar,Yoga Studio,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Donut Shop
2,2,M1E,Scarborough,"Guildwood,Morningside,West Hill",M1E,43.763573,-79.188711,2,Electronics Store,Medical Center,Mexican Restaurant,Spa,Breakfast Spot,Pizza Place,Rental Car Location,Intersection,Dim Sum Restaurant,Diner
3,3,M1G,Scarborough,Woburn,M1G,43.770992,-79.216917,2,Coffee Shop,Pharmacy,Korean Restaurant,Yoga Studio,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant
4,4,M1H,Scarborough,Cedarbrae,M1H,43.773136,-79.239476,2,Gas Station,Bakery,Fried Chicken Joint,Caribbean Restaurant,Athletics & Sports,Thai Restaurant,Bank,Hakka Restaurant,Dumpling Restaurant,Drugstore


### Finally, let's visualize the resulting clusters

In [160]:

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
print('x:', x)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
print('ys:', ys)
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
#print(colors_array)
rainbow = [colors.rgb2hex(i) for i in colors_array]
print(rainbow)

# add markers to the map
markers_colors = []
for lat, lon, nb, cluster in zip(Toronto_merged['Latitude'], Toronto_merged['Longitude'], Toronto_merged['Neighbourhood'], Toronto_merged['Cluster Labels']):
    label = folium.Popup(str(nb) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

x: [0 1 2 3 4]
ys: [array([0, 1, 2, 3, 4]), array([ 1,  3,  7, 13, 21]), array([ 2,  7, 20, 41, 70]), array([  3,  13,  41,  87, 151]), array([  4,  21,  70, 151, 264])]
['#8000ff', '#00b5eb', '#80ffb4', '#ffb360', '#ff0000']


### Examining Cluster 1

In [161]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 0, Toronto_merged.columns[[3] + list(range(7, Toronto_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
21,Willowdale South,0,Coffee Shop,Ramen Restaurant,Café,Pizza Place,Japanese Restaurant,Restaurant,Sushi Restaurant,Sandwich Place,Grocery Store,Discount Store
37,Leaside,0,Coffee Shop,Sporting Goods Shop,Furniture / Home Store,Burger Joint,Grocery Store,Breakfast Spot,Fish & Chips Shop,Sports Bar,Beer Store,Smoothie Shop
40,"The Danforth West,Riverdale",0,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Restaurant,Bookstore,Furniture / Home Store,Fruit & Vegetable Store,Pub,Pizza Place
42,Studio District,0,Café,Coffee Shop,Bakery,Italian Restaurant,American Restaurant,Neighborhood,Seafood Restaurant,Brewery,Sandwich Place,Cheese Shop
46,Davisville,0,Sandwich Place,Dessert Shop,Pizza Place,Café,Coffee Shop,Sushi Restaurant,Gym,Italian Restaurant,Farmers Market,Diner
50,"Cabbagetown,St. James Town",0,Coffee Shop,Restaurant,Flower Shop,Bakery,Italian Restaurant,Pub,Café,Pizza Place,Grocery Store,Pet Store
51,Church and Wellesley,0,Sushi Restaurant,Coffee Shop,Gay Bar,Japanese Restaurant,Restaurant,Café,Hotel,Mediterranean Restaurant,Gym,Gastropub
52,Harbourfront,0,Coffee Shop,Park,Bakery,Pub,Restaurant,Breakfast Spot,Café,Mexican Restaurant,Theater,Dessert Shop
55,Berczy Park,0,Coffee Shop,Farmers Market,Café,Bakery,Steakhouse,Cocktail Bar,Beer Bar,Cheese Shop,Seafood Restaurant,Restaurant
61,"Bedford Park,Lawrence Manor East",0,Sushi Restaurant,Coffee Shop,Italian Restaurant,Grocery Store,Butcher,Café,Indian Restaurant,Restaurant,Hobby Shop,Fast Food Restaurant


### Examining Cluster 2

In [163]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] ==1 , Toronto_merged.columns[[3] + list(range(7, Toronto_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
54,St. James Town,1,Café,Coffee Shop,Restaurant,Clothing Store,Hotel,Cosmetics Shop,American Restaurant,Beer Bar,Italian Restaurant,Diner
56,Central Bay Street,1,Coffee Shop,Sandwich Place,Italian Restaurant,Burger Joint,Café,Ice Cream Shop,Bar,Juice Bar,Bubble Tea Shop,Chinese Restaurant
57,"Adelaide,King,Richmond",1,Coffee Shop,Café,Thai Restaurant,Steakhouse,Bar,Bakery,Restaurant,Salad Place,Sushi Restaurant,Asian Restaurant
58,"Harbourfront East,Toronto Islands,Union Station",1,Coffee Shop,Aquarium,Café,Hotel,Fried Chicken Joint,Scenic Lookout,Brewery,Restaurant,Italian Restaurant,Park
59,"Design Exchange,Toronto Dominion Centre",1,Coffee Shop,Café,Hotel,Restaurant,Bar,American Restaurant,Deli / Bodega,Seafood Restaurant,Bakery,Gastropub
60,"Commerce Court,Victoria Hotel",1,Coffee Shop,Café,Hotel,Restaurant,American Restaurant,Deli / Bodega,Italian Restaurant,Gym,Bakery,Gastropub
68,Stn A PO Boxes 25 The Esplanade,1,Coffee Shop,Café,Restaurant,Italian Restaurant,Japanese Restaurant,Beer Bar,Seafood Restaurant,Hotel,Art Gallery,Cocktail Bar
69,"First Canadian Place,Underground city",1,Coffee Shop,Café,Restaurant,Steakhouse,Hotel,Asian Restaurant,American Restaurant,Burger Joint,Bar,Bakery


### Examining Cluster 3

In [164]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 2, Toronto_merged.columns[[3] + list(range(7, Toronto_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Rouge,Malvern",2,Fast Food Restaurant,Print Shop,Yoga Studio,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Donut Shop
1,"Highland Creek,Rouge Hill,Port Union",2,Moving Target,Construction & Landscaping,Bar,Yoga Studio,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Donut Shop
2,"Guildwood,Morningside,West Hill",2,Electronics Store,Medical Center,Mexican Restaurant,Spa,Breakfast Spot,Pizza Place,Rental Car Location,Intersection,Dim Sum Restaurant,Diner
3,Woburn,2,Coffee Shop,Pharmacy,Korean Restaurant,Yoga Studio,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant
4,Cedarbrae,2,Gas Station,Bakery,Fried Chicken Joint,Caribbean Restaurant,Athletics & Sports,Thai Restaurant,Bank,Hakka Restaurant,Dumpling Restaurant,Drugstore
5,Scarborough Village,2,Playground,Yoga Studio,Donut Shop,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Drugstore
6,"East Birchmount Park,Ionview,Kennedy Park",2,Discount Store,Hobby Shop,Convenience Store,Bus Station,Chinese Restaurant,Department Store,Coffee Shop,Eastern European Restaurant,Electronics Store,Dumpling Restaurant
7,"Clairlea,Golden Mile,Oakridge",2,Bakery,Bus Line,Soccer Field,Metro Station,Bus Station,Fast Food Restaurant,Intersection,Park,Gastropub,Deli / Bodega
8,"Cliffcrest,Cliffside,Scarborough Village West",2,American Restaurant,Motel,Deli / Bodega,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Yoga Studio
9,"Birch Cliff,Cliffside West",2,Café,College Stadium,Skating Rink,General Entertainment,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run


### Examining Cluster 4

In [165]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 3, Toronto_merged.columns[[3] + list(range(7, Toronto_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
66,"Chinatown,Grange Park,Kensington Market",3,Café,Bar,Vietnamese Restaurant,Dumpling Restaurant,Mexican Restaurant,Chinese Restaurant,Bakery,Coffee Shop,Vegetarian / Vegan Restaurant,Grocery Store


### Examining Cluster 5

In [166]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 4, Toronto_merged.columns[[3] + list(range(7, Toronto_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
17,"Fairview,Henry Farm,Oriole",4,Clothing Store,Coffee Shop,Fast Food Restaurant,Women's Store,Tea Room,Bakery,Food Court,Japanese Restaurant,Bus Station,Shopping Mall
53,"Ryerson,Garden District",4,Coffee Shop,Clothing Store,Cosmetics Shop,Café,Middle Eastern Restaurant,Fast Food Restaurant,Tea Room,Italian Restaurant,Bubble Tea Shop,Pizza Place
